In [6]:
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from numpy import char as npc
import requests
from bs4 import BeautifulSoup
import scipy.stats
import matplotlib.pyplot as plt
import re
import os
import math
from datetime import datetime
%matplotlib inline

import seaborn as sns
sns.set()

import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')

The following jupyter notebook illustrate the step tooken to scrape required real estate offers from [tutti.ch](https://www.tutti.ch/fr/li/vaud/lausanne/immobilier?o=1).

For our work we only need the annoncements of real estate in the area of Lausanne.

The final dataframe is saved as a JSON file named 'tutti.json'.

In [19]:
RAW_DIR = '../../data/raw/tutti/'
FINAL_DIR = '../../data/rents/'

### 1. Go through all pages and save it locally

The first step is go through all the available pages and save the response JSON locally. The data are saved in the `data/raw/tutti` directory.

In [8]:
def getTutti(page, limit=100):
    cookies = {
        'ajs_user_id': 'null',
        'ajs_group_id': 'null',
        'lang': 'fr',
        'ajs_anonymous_id': '%22bf0a3c34-c83f-4764-a48f-c783309a9f4c%22',
        '_gcl_au': '1.1.253718592.1542968730',
        '_ga': 'GA1.2.1357180133.1542968730',
        '_gid': 'GA1.2.2109516043.1542968730',
        'adw': '12a92c4c-f201-44a3-a5ce-26dbbce57a94',
        '_gat_UA-88671020-1': '1',
    }
    headers = {
        'AlexaToolbar-ALX_NS_PH': 'AlexaToolbar/alx-4.0.3',
        'Origin': 'https://www.tutti.ch',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'fr',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Referer': 'https://www.tutti.ch/fr/li/vaud/lausanne/immobilier/appartements?o=2',
        'Connection': 'keep-alive',
        'X-Tutti-Hash': '81fd68a0-47a0-4b8b-9401-4c63cb416c5e',
        'X-Tutti-Source': 'web LIVE-181123-163',
        'DNT': '1',
    }
    params = (
        ('category', '1000'),
        ('limit', limit),
        ('m', '131'),
        ('o', page),
        ('region', '20'), # Region 20: lausanne
        ('subcategory', ''),
        ('with_all_regions', 'false'),
    )

    response = requests.get('https://api.tutti.ch/v10/list.json', headers=headers, params=params, cookies=cookies)
    return json.loads(response.content)

In [9]:
def saveJson(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)

In [10]:
def loadJson(filename):
    with open(filename, 'r') as outfile:
        return json.load(outfile)

Get all tutti.ch advertisment from region 20

In [20]:
# Compute number of pages to iterate
limit = 100
total_ads = getTutti(page=1, limit=limit)['search_total']
pages = math.ceil(total_ads/limit)

# Iterate through all pages in region=20 (Lausanne) and save data locally
for page in np.arange(1, pages+1):
    j = getTutti(page=page, limit=limit)
    filename = RAW_DIR + 'page_' + str(page) + '.json'
    saveJson(j, filename)

At this point there are *pages* different JSON file that need to be read, merged and cleaned.

### 2. Open, merge and clean

Iterate over all *json* in dir and save it on a JSON. 

The `json_normalize` function turned out to be very useful to flatten all JSON cells. 

In [21]:
all_json_filename = os.listdir(RAW_DIR)
# Take only .json file
all_json_filename = list(filter(lambda x: x.endswith('.json'), all_json_filename))

all_json = []

for j_ in all_json_filename:
    print(j_)
    j_tmp = loadJson(RAW_DIR + j_)
    
    # Read JSON (and flatten values)
    pd_ = json_normalize(j_tmp['items'])
    
    # Save pandas obj 
    all_json.append(pd_)
    
df = pd.concat(all_json, ignore_index=True ,sort=False)

page_1.json
page_2.json
page_3.json
page_4.json
page_5.json
page_6.json
page_7.json


In [22]:
df.head(2)

,body,category_info.id,category_info.name,category_info.parent_id,category_info.parent_name,company_ad,epoch_time,highlight,id,image_names,...,location_info.region_id,location_info.region_name,parameters,phone_hash,price,public_account_id,subject,thumb_name,type,user_alias
0,Sur deux niveaux. Quatre chambres Ã l'Ã©tage....,1040,Maisons,1000,Immobilier,True,1542978625,False,24826554,"[5802942370.jpg, 5854082284.jpg, 5817113187.jp...",...,20,Vaud,"[{'id': 'price', 'value': '1'580'000.-', 'labe...",fc1a3809531909a7d549eba30d44ff6b38e21dba,1'580'000.-,NaN,Belle maison moderne et confortable Ã vendre,5802942370.jpg,s,JustImmo
1,"Ã€ louer au Boveresses 73 1010 lausanne , \nun...",1020,Appartements,1000,Immobilier,True,1542977489,False,23588353,[],...,20,Vaud,"[{'id': 'price', 'value': '1'470.- par mois', ...",9f0ff2bc2a269be5e044139dd6688148d0dea247,1'470.- par mois,NaN,Appartement Ã louer,NaN,u,ImmoArena24


In [23]:
df.shape

(676, 25)

The DataFrame df still contains a column 'parameters' that is not an easy parsable JSON object. Get rid of it in a dirty way:

In [24]:
def posIdInParameter(id_, p):
    """Return position of the list where the id is contained"""
    pos = -1
    values = [value['id'] for value in p]
    zip_ = dict(zip(values, np.arange(0,len(p))))
    if (id_ in values):
        pos = zip_[id_]
    return pos

In [25]:
def cleanParameter(p):
    """From parameter objects p, return Pandas Series with only nmr. of rooms, size and type"""
    
    rooms, size, type_ = [np.nan] * 3
    
    # Check for rooms
    pos_rooms = posIdInParameter('rooms', p)
    if (pos_rooms != -1):
        rooms = p[pos_rooms]['value']
        
    # Check for size
    pos_size = posIdInParameter('size', p)
    if (pos_size != -1):
        size = p[pos_size]['value']
        
    # Check for type_
    pos_type = posIdInParameter('type', p)
    if (pos_type != -1):
        type_ = p[pos_type]['value']
    
    return pd.Series([rooms, size, type_])

In [26]:
parameters = df['parameters'].apply(lambda p: cleanParameter(p))
parameters.columns = ['rooms', 'size', 'type_param']

In [27]:
df = df.merge(parameters, left_index=True, right_index=True)


# Drop not useful columns
not_useful_columns = ['company_ad', 
                      'image_names', 
                      'language', 
                      'parameters', 
                      'phone_hash', 
                      'thumb_name', 
                      'category_info.id',
                      'category_info.parent_id',
                      'category_info.parent_name',
                      'highlight',
                      'location_info.area', # since all 'lausanne',
                      'location_info.area_id',
                      'location_info.region_name',
                      'location_info.region_id',
                      'public_account_id',
                     ]
df.drop(not_useful_columns, axis='columns', inplace=True)

df.head(2)

,body,category_info.name,epoch_time,id,location_info.address,location_info.plz,price,subject,type,user_alias,rooms,size,type_param
0,Sur deux niveaux. Quatre chambres Ã l'Ã©tage....,Maisons,1542978625,24826554,NaN,1066,1'580'000.-,Belle maison moderne et confortable Ã vendre,s,JustImmo,5.5,170m²,Offres de vente
1,"Ã€ louer au Boveresses 73 1010 lausanne , \nun...",Appartements,1542977489,23588353,NaN,1010,1'470.- par mois,Appartement Ã louer,u,ImmoArena24,3.5,90m²,Offres de location


### 3. Export to JSON format

In [31]:
# Rename columns
df.rename({'subject':'title', 
                'rooms': 'numberRooms', 
                'location_info.address': 'address',
                'location_info.plz': 'postCode',
                'size':'surface',
               }, axis='columns', inplace=True)

df.head(2)

,body,category_info.name,epoch_time,id,address,postCode,price,title,type,user_alias,numberRooms,surface,type_param
0,Sur deux niveaux. Quatre chambres Ã l'Ã©tage....,Maisons,1542978625,24826554,NaN,1066,1'580'000.-,Belle maison moderne et confortable Ã vendre,s,JustImmo,5.5,170m²,Offres de vente
1,"Ã€ louer au Boveresses 73 1010 lausanne , \nun...",Appartements,1542977489,23588353,NaN,1010,1'470.- par mois,Appartement Ã louer,u,ImmoArena24,3.5,90m²,Offres de location


Export to JSON format:

In [35]:
df.to_json(FINAL_DIR + 'tutti.json')